In [243]:
import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from typing import Final
from tqdm.notebook import tqdm
from __future__ import annotations

In [244]:
PLAYER_NAME_COL: Final[int] = 7
STARTING_DATE_COL: Final[int] = 0
LABELENCODER_COLUMNS: Final[tuple[str]] = (
    '開催場所',	'グレード',	'レース名', '性別', '級班', '脚質'
)
LABELENCODER_TRANSFORM_COLUMNS: Final[tuple[str]] = (
    'レース1_グレード', 'レース2_グレード', 'レース3_グレード', 'レース4_グレード', 'レース5_グレード'
)
DF_MERGE_COLUMNS: Final[tuple[str]] = (
    '発走日時','開催場所','グレード','レースグループ','レース名','ラウンド','年度','選手名','枠番','車番','1位フラグ','1位2位フラグ','性別','級班','脚質',
    '登録番号','年齢','身長','体重','1着','2着','3着','4着～','出走回数','優勝回数','勝率','2連対率','3連対率',
    'レース1_レース開始日','レース1_レース名','レース1_グレード','レース1_着順1','レース1_着順2','レース1_着順3','レース1_着順4','レース1_着順5',
    'レース2_レース開始日','レース2_レース名','レース2_グレード','レース2_着順1','レース2_着順2','レース2_着順3','レース2_着順4','レース2_着順5',
    'レース3_レース開始日','レース3_レース名','レース3_グレード','レース3_着順1','レース3_着順2','レース3_着順3','レース3_着順4','レース3_着順5',
    'レース4_レース開始日','レース4_レース名','レース4_グレード','レース4_着順1','レース4_着順2','レース4_着順3','レース4_着順4','レース4_着順5',
    'レース5_レース開始日','レース5_レース名','レース5_グレード','レース5_着順1','レース5_着順2','レース5_着順3','レース5_着順4','レース5_着順5'
)

In [245]:
# def labelencoder(df: pd.DataFrame) -> pd.DataFrame:
#     for column in LABELENCODER_COLUMNS:
#         le = LabelEncoder()
#         le.fit(df[column])
#         df[column] = le.transform(df[column])

#     return df

In [246]:
def read_pickle_to_df() -> tuple[pd.DataFrame]:
    """
    各種pickleファイルを読み込み、各種データフレームを返却する

    Returns
    -------
    tuple[pd.DataFrame]
        各種データフレームのタプル
    """
    df_player_results = pd.read_pickle('player_results.pkl')
    df_yearly_results = pd.read_pickle('yearly_results.pkl')
    df_race_results = pd.read_pickle('race_results.pkl')
    df_participants = pd.read_pickle('participants.pkl')
    df_player_basic_data = pd.read_pickle('player_basic_data.pkl')

    return df_player_results, df_yearly_results, df_race_results, df_participants, df_player_basic_data

In [247]:
# def yearly_results_merge(df_m: pd.DataFrame, df_y: pd.DataFrame) -> pd.DataFrame:

#     df_list = []
#     df_key = df_m['発走年度']
#     for i, starting_year in enumerate(df_key):
#         player_name = df_m['選手名'][i]
#         df_temp = df_y.query('選手名==@player_name and 年度<@starting_year')
#         df_temp = df_temp.groupby('選手名').agg(['sum', 'mean'])
#         df_temp = df_temp[[('1着','sum'), ('2着', 'sum'), ('3着', 'sum'), ('4着〜', 'sum'),
#                            ('出走回数', 'sum'), ('優勝回数', 'sum'), ('勝率', 'mean'),
#                            ('2連対率', 'mean'), ('3連対率', 'mean')]]
#         df_temp.columns = ('1着', '2着', '3着', '4着～', '出走回数', '優勝回数', '勝率', '2連対率', '3連対率')
#         df_list.append(df_temp)
#     df_temp = pd.concat(df_list)
#     df_m = pd.concat([df_m, df_temp], axis=1)


In [248]:
def player_results_merge(df_m: pd.DataFrame, df_p: pd.DataFrame) -> pd.DataFrame:
    """
    選手の直近5試合の試合結果を

    Parameters
    ----------
    df_m : pd.DataFrame
        [description]
    df_p : pd.DataFrame
        [description]

    Returns
    -------
    pd.DataFrame
        [description]
    """
    
    df_list = []
    for i in tqdm(range(len(df_m)), desc='player_results_merge'):
        player_name = df_m.iat[i, PLAYER_NAME_COL]
        starting_date = df_m.iat[i, STARTING_DATE_COL]
        starting_date -= datetime.timedelta(days=1)
        df_extract_datas = df_p.query('選手名==@player_name and レース開始日<=@starting_date').head()
        df_for_join = pd.DataFrame()
        for n in range(len(df_extract_datas)):
            df_temp = df_extract_datas.iloc[[n], :]
            columns = ['レース' + str(n + 1) + '_' + column for column in df_temp]
            df_temp.columns = columns
            df_for_join = pd.concat([df_for_join, df_temp], axis=1)
        df_for_merge = df_m.iloc[[i], :]
        try:
            df_for_merge = pd.merge(df_for_merge, df_for_join, on='選手名', how='left')
        except KeyError:
            df_for_join = pd.DataFrame(np.full(40, np.nan))
            df_for_merge = pd.concat([df_for_merge, df_for_join], axis=1)
        df_list.append(df_for_merge)

    df = pd.concat(df_list)
    return df

In [249]:
# def player_results_merge(df_m: pd.DataFrame, df_p: pd.DataFrame) -> pd.DataFrame:
#     np_merge_area = None
#     for i in tqdm(range(len(df_m)), desc='player_results_merge'):
#         player_name = df_m.iat[i, PLAYER_NAME_COL]
#         starting_date = df_m.iat[i, STARTING_DATE_COL]
#         starting_date -= datetime.timedelta(days=1)
#         np_extract_datas = df_p.query('選手名==@player_name and レース開始日<=@starting_date').head().values
#         if np_extract_datas.shape[0] <= 5:
#             loop_num = 5 - np_extract_datas.shape[0]
#             for _ in range(loop_num):
#                 np_extract_datas = np.vstack([np_extract_datas, np.full(8, np.nan)])
#         np_for_join = None
#         for n in range(len(np_extract_datas)):
#             np_temp = np_extract_datas[n]
#             if n == 0:
#                 np_for_join = np_temp
#             else:
#                 np_for_join = np.hstack([np_for_join, np_temp])
#         np_for_merge = df_m.iloc[i, :].values
#         np_for_merge = np.hstack([np_for_merge, np_for_join])
#         if i == 0:
#             np_merge_area = np_for_merge
#         else:
#             np_merge_area = np.vstack([np_merge_area, np_for_merge])

#     df = pd.DataFrame(np_merge_area.reshape(1, -1))
#     df.columns = DF_MERGE_COLUMNS
#     return df

In [250]:
df_player_results, df_yearly_results, df_race_results, df_participants, df_player_basic_data = read_pickle_to_df()
df_merge = pd.merge(df_participants, df_player_basic_data, on='選手名', how='left')
df_merge = pd.merge(df_merge, df_yearly_results, on=['選手名', '年度'], how='left')
df_player_results = df_player_results.set_index('選手名')
df_merge = df_merge.sort_values(['発走日時', '開催場所', '車番'])
df_merge = player_results_merge(df_merge, df_player_results)


KeyboardInterrupt: 

In [ ]:
pd.set_option('display.max_columns', 100)
df_merge = df_merge.sort_values(['発走日時', '開催場所', '車番'])
df_merge.query('発走日時=="2017-01-09 15:16:00"')

,発走日時,開催場所,グレード,レースグループ,レース名,ラウンド,年度,選手名,枠番,車番,1位フラグ,1位2位フラグ,性別,級班,脚質,登録番号,年齢,身長,体重,1着,2着,3着,4着～,出走回数,優勝回数,勝率,2連対率,3連対率
841631,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,佐藤正吾,1,1,0,0,男,A級3班,追,13907.0,38,170.0,70.0,17.0,33.0,45.0,154.0,249.0,0.0,6.700000,19.833333,38.100000
841632,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,近藤悠人,2,2,0,0,男,A級3班,両,14346.0,34,177.0,81.0,56.0,27.0,20.0,161.0,264.0,7.0,20.766667,30.733333,38.166667
841633,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,大石剣士,3,3,1,1,男,S級1班,逃,15096.0,25,172.1,69.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
841634,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,菊地大輔,4,4,0,1,男,A級3班,追,13677.0,46,180.0,77.0,9.0,21.0,27.0,145.0,202.0,0.0,4.233333,13.333333,26.466667
841635,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,松中宏樹,5,5,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
841636,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,吉田雄三,6,6,0,0,男,A級3班,追,13085.0,47,170.0,75.0,3.0,11.0,26.0,204.0,244.0,0.0,1.233333,5.766667,16.566667
841637,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,島村健吉,6,7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_participants.query('発走日時=="2017-01-09 15:16:00"')

,発走日時,開催場所,グレード,レースグループ,レース名,ラウンド,年度,選手名,枠番,車番,1位フラグ,1位2位フラグ
534,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,佐藤正吾,1,1,0,0
533,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,近藤悠人,2,2,0,0
531,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,大石剣士,3,3,1,1
532,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,菊地大輔,4,4,0,1
535,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,松中宏樹,5,5,0,0
536,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,吉田雄三,6,6,0,0
537,2017-01-09 15:16:00,松山,FII,愛媛新聞社杯 松山競輪誕生記念,Ａ級 チャレンジ予選,1,2015,島村健吉,6,7,0,0


In [ ]:
df_player_results.query('選手名=="松中宏樹"')

,選手名,レース開始日,レース名,グレード,着順1,着順2,着順3,着順4,着順5
